In [89]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

## Create the market basket dataset and use one-hot encoding for items

In [2]:
dataset = MarketBasketDataset(dataset_path=r"C:\Users\eitan\PycharmProjects\EntangledExplainableClustering\data\Groceries_dataset.csv")

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=32)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        knn_loss = knn_crt(iterm)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.115936279296875 | KNN Loss: 6.224985599517822 | BCE Loss: 1.8909504413604736
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.152900695800781 | KNN Loss: 6.225427150726318 | BCE Loss: 1.927473783493042
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.18317985534668 | KNN Loss: 6.225057125091553 | BCE Loss: 1.9581222534179688
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.14844799041748 | KNN Loss: 6.224520683288574 | BCE Loss: 1.9239273071289062
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.107247352600098 | KNN Loss: 6.224483489990234 | BCE Loss: 1.8827638626098633
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.133842468261719 | KNN Loss: 6.224823474884033 | BCE Loss: 1.9090194702148438
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.103509902954102 | KNN Loss: 6.223139762878418 | BCE Loss: 1.880369782447815
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.12332534790039 | KNN Loss: 6.223299980163574 | BCE Loss: 1.9000256061

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 5.691650390625 | KNN Loss: 4.585964202880859 | BCE Loss: 1.1056861877441406
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.657222747802734 | KNN Loss: 4.540197849273682 | BCE Loss: 1.1170251369476318
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.559140682220459 | KNN Loss: 4.475207328796387 | BCE Loss: 1.0839334726333618
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 5.476250648498535 | KNN Loss: 4.389369010925293 | BCE Loss: 1.0868817567825317
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.420239448547363 | KNN Loss: 4.333858013153076 | BCE Loss: 1.0863815546035767
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 5.372244834899902 | KNN Loss: 4.272647857666016 | BCE Loss: 1.0995970964431763
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.269759178161621 | KNN Loss: 4.20658540725708 | BCE Loss: 1.0631736516952515
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.299194812774658 | KNN Loss: 4.192719459533691 | BCE Loss: 1.

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.917761325836182 | KNN Loss: 3.854158878326416 | BCE Loss: 1.0636024475097656
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.8506550788879395 | KNN Loss: 3.80037522315979 | BCE Loss: 1.0502798557281494
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.865542888641357 | KNN Loss: 3.8176093101501465 | BCE Loss: 1.0479336977005005
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.878241539001465 | KNN Loss: 3.829946279525757 | BCE Loss: 1.0482953786849976
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.888167858123779 | KNN Loss: 3.831289768218994 | BCE Loss: 1.0568782091140747
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.8718719482421875 | KNN Loss: 3.817835569381714 | BCE Loss: 1.0540366172790527
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.849645614624023 | KNN Loss: 3.808359146118164 | BCE Loss: 1.0412867069244385
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 4.8606719970703125 | KNN Loss: 3.8155460357666016 | BCE

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.803418159484863 | KNN Loss: 3.7790985107421875 | BCE Loss: 1.0243196487426758
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.8621320724487305 | KNN Loss: 3.814321994781494 | BCE Loss: 1.0478098392486572
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.788928031921387 | KNN Loss: 3.738187551498413 | BCE Loss: 1.0507407188415527
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.772955894470215 | KNN Loss: 3.7529213428497314 | BCE Loss: 1.020034670829773
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.833101749420166 | KNN Loss: 3.7818121910095215 | BCE Loss: 1.051289439201355
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.817929267883301 | KNN Loss: 3.795948028564453 | BCE Loss: 1.0219810009002686
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.77934455871582 | KNN Loss: 3.7568111419677734 | BCE Loss: 1.0225335359573364
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 4.878941535949707 | KNN Loss: 3.80889630317688 | BCE Lo

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.8286662101745605 | KNN Loss: 3.7770891189575195 | BCE Loss: 1.0515769720077515
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.798503398895264 | KNN Loss: 3.756279706954956 | BCE Loss: 1.042223572731018
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.760849952697754 | KNN Loss: 3.7435646057128906 | BCE Loss: 1.0172855854034424
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.74405574798584 | KNN Loss: 3.736921548843384 | BCE Loss: 1.007134199142456
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.801478862762451 | KNN Loss: 3.750027894973755 | BCE Loss: 1.0514508485794067
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.815277099609375 | KNN Loss: 3.7805542945861816 | BCE Loss: 1.0347228050231934
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.7980265617370605 | KNN Loss: 3.7462282180786133 | BCE Loss: 1.0517983436584473
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 4.757270336151123 | KNN Loss: 3.745525360107422 | BCE 

...

Epoch 448 / 500 | iteration 0 / 30 | Total Loss: 4.732075214385986 | KNN Loss: 3.7153801918029785 | BCE Loss: 1.0166950225830078
Epoch 448 / 500 | iteration 5 / 30 | Total Loss: 4.716768264770508 | KNN Loss: 3.6851818561553955 | BCE Loss: 1.0315861701965332
Epoch 448 / 500 | iteration 10 / 30 | Total Loss: 4.7052998542785645 | KNN Loss: 3.690096616744995 | BCE Loss: 1.0152032375335693
Epoch 448 / 500 | iteration 15 / 30 | Total Loss: 4.742417335510254 | KNN Loss: 3.6937782764434814 | BCE Loss: 1.0486392974853516
Epoch 448 / 500 | iteration 20 / 30 | Total Loss: 4.696242809295654 | KNN Loss: 3.697507619857788 | BCE Loss: 0.9987351298332214
Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 4.72514009475708 | KNN Loss: 3.6809780597686768 | BCE Loss: 1.0441621541976929
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.748376846313477 | KNN Loss: 3.7178196907043457 | BCE Loss: 1.0305571556091309
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.729743480682373 | KNN Loss: 3.70274162292480

Epoch 458 / 500 | iteration 20 / 30 | Total Loss: 4.7442216873168945 | KNN Loss: 3.72982120513916 | BCE Loss: 1.014400601387024
Epoch 458 / 500 | iteration 25 / 30 | Total Loss: 4.720094680786133 | KNN Loss: 3.694822311401367 | BCE Loss: 1.025272250175476
Epoch 459 / 500 | iteration 0 / 30 | Total Loss: 4.7125420570373535 | KNN Loss: 3.689485788345337 | BCE Loss: 1.0230562686920166
Epoch 459 / 500 | iteration 5 / 30 | Total Loss: 4.674315929412842 | KNN Loss: 3.6623287200927734 | BCE Loss: 1.011987328529358
Epoch 459 / 500 | iteration 10 / 30 | Total Loss: 4.732183456420898 | KNN Loss: 3.7102231979370117 | BCE Loss: 1.0219604969024658
Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 4.6692352294921875 | KNN Loss: 3.6817195415496826 | BCE Loss: 0.987515926361084
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.71513557434082 | KNN Loss: 3.6937007904052734 | BCE Loss: 1.021435022354126
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.742697238922119 | KNN Loss: 3.6982126235961914 

Epoch 469 / 500 | iteration 10 / 30 | Total Loss: 4.695771217346191 | KNN Loss: 3.6765801906585693 | BCE Loss: 1.019190788269043
Epoch 469 / 500 | iteration 15 / 30 | Total Loss: 4.709070205688477 | KNN Loss: 3.6821446418762207 | BCE Loss: 1.0269253253936768
Epoch 469 / 500 | iteration 20 / 30 | Total Loss: 4.658243656158447 | KNN Loss: 3.664552927017212 | BCE Loss: 0.9936908483505249
Epoch 469 / 500 | iteration 25 / 30 | Total Loss: 4.726993560791016 | KNN Loss: 3.6653761863708496 | BCE Loss: 1.0616176128387451
Epoch 470 / 500 | iteration 0 / 30 | Total Loss: 4.704527378082275 | KNN Loss: 3.685822010040283 | BCE Loss: 1.0187053680419922
Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 4.696495056152344 | KNN Loss: 3.66705584526062 | BCE Loss: 1.0294393301010132
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.683419704437256 | KNN Loss: 3.684457540512085 | BCE Loss: 0.9989620447158813
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.679872035980225 | KNN Loss: 3.6961405277252197

Epoch 480 / 500 | iteration 0 / 30 | Total Loss: 4.680174350738525 | KNN Loss: 3.695300579071045 | BCE Loss: 0.9848736524581909
Epoch 480 / 500 | iteration 5 / 30 | Total Loss: 4.6911163330078125 | KNN Loss: 3.6690118312835693 | BCE Loss: 1.0221046209335327
Epoch 480 / 500 | iteration 10 / 30 | Total Loss: 4.690283298492432 | KNN Loss: 3.699307680130005 | BCE Loss: 0.9909757971763611
Epoch 480 / 500 | iteration 15 / 30 | Total Loss: 4.677189826965332 | KNN Loss: 3.6597557067871094 | BCE Loss: 1.0174338817596436
Epoch 480 / 500 | iteration 20 / 30 | Total Loss: 4.69784688949585 | KNN Loss: 3.669581890106201 | BCE Loss: 1.0282648801803589
Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 4.691289901733398 | KNN Loss: 3.673945188522339 | BCE Loss: 1.0173447132110596
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.698472023010254 | KNN Loss: 3.668053150177002 | BCE Loss: 1.030419111251831
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.7260236740112305 | KNN Loss: 3.7319328784942627 

Epoch 490 / 500 | iteration 20 / 30 | Total Loss: 4.658114910125732 | KNN Loss: 3.671670913696289 | BCE Loss: 0.9864441752433777
Epoch 490 / 500 | iteration 25 / 30 | Total Loss: 4.715171813964844 | KNN Loss: 3.6964666843414307 | BCE Loss: 1.018704891204834
Epoch 491 / 500 | iteration 0 / 30 | Total Loss: 4.735581398010254 | KNN Loss: 3.679997682571411 | BCE Loss: 1.0555838346481323
Epoch 491 / 500 | iteration 5 / 30 | Total Loss: 4.728246212005615 | KNN Loss: 3.735576629638672 | BCE Loss: 0.9926697015762329
Epoch 491 / 500 | iteration 10 / 30 | Total Loss: 4.7432451248168945 | KNN Loss: 3.7044808864593506 | BCE Loss: 1.038764238357544
Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 4.702518463134766 | KNN Loss: 3.6970551013946533 | BCE Loss: 1.0054631233215332
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.74367094039917 | KNN Loss: 3.6656463146209717 | BCE Loss: 1.0780247449874878
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.733044624328613 | KNN Loss: 3.684960603713989

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 1.7604,  4.4755,  3.0771,  3.9097,  4.0269,  0.5695,  1.7009,  2.7212,
          1.9785,  1.8334,  2.6973,  2.0479,  0.5888,  2.0219,  1.4895,  0.5669,
          1.7798,  2.0688,  3.1418,  1.2685,  2.0295,  1.9587,  1.8234,  2.0338,
          1.7594,  1.3992,  2.6015,  0.9626,  1.8034,  0.1201, -0.0493,  0.8887,
          0.1603,  0.8327,  1.9717,  1.0101,  0.7896,  1.8603,  0.6187,  1.4763,
          0.8109, -0.5784, -0.3982,  2.8505,  1.4647,  0.7623, -0.0900,  0.2955,
          1.6988,  2.8949,  2.0963,  0.3493,  1.5421,  0.9165, -0.3905,  0.9805,
          1.5791,  1.6771,  1.2341,  1.6186,  0.4137,  0.5113, -0.0765,  1.7783,
          0.6330,  1.9503, -1.7280,  0.0204,  1.8930,  2.5013,  2.9008, -0.0242,
          1.5629,  2.4864,  2.0565,  1.5185,  0.1033,  0.7187, -0.1259,  1.9349,
          0.0165, -0.0965,  2.1189, -0.2565,  0.5036, -1.0598, -2.2981, -0.0859,
          0.8270, -1.8429,  0.0118, -0.0174, -0.5732, -0.6276,  0.3815,  1.0661,
         -0.6287, -0.8592,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0] for d in dataset]

In [11]:
projections = model.calculate_intermidiate(dataset_)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:03<00:00,  4.22it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [63]:
clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
print(clf.get_depth())

0.22159491884262528
132


In [62]:
scores = []
for min_samples in range(1,50, 1):
    clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
    clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
    scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
plt.figure()
plt.plot(list(range(1,50, 1)), scores)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [65]:
rules = get_rules(clf, dataset.items, clusters[clusters != -1])

for rule in rules:
    n_pos = 0
    for c,p,v in rule['rule']:
        if p == '>':
            n_pos += 1
    rule['pos'] = n_pos

In [66]:
plt.figure()
probs = [r['proba'] for r in rules]
plt.hist(probs, bins = 100)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
rules = sorted(rules, key=lambda x:x['pos'])
rules = [r for r in rules if r['proba'] > 50]
print(len(rules))

17


In [69]:
for i in range(17):
    r_i = rules[i]
    print(f"------------- rule {i} length {len(r_i)} -------------")
    print(r_i)

------------- rule 0 length 4 -------------
{'target': ' then class: 3 (proba: 55.56%) | based on 9 samples', 'rule': [('oil', '<=', 0.5), ('ketchup', '<=', 0.5), ('sliced cheese', '<=', 0.5), ('dish cleaner', '<=', 0.5), ('sparkling wine', '<=', 0.5), ('dental care', '<=', 0.5), ('specialty chocolate', '<=', 0.5), ('bottled beer', '<=', 0.5), ('pastry', '<=', 0.5), ('pasta', '<=', 0.5), ('liqueur', '<=', 0.5), ('frozen fruits', '<=', 0.5), ('yogurt', '<=', 0.5), ('mayonnaise', '<=', 0.5), ('processed cheese', '<=', 0.5), ('fish', '<=', 0.5), ('berries', '<=', 0.5), ('soap', '<=', 0.5), ('hamburger meat', '<=', 0.5), ('onions', '<=', 0.5), ('rum', '<=', 0.5), ('frozen dessert', '<=', 0.5), ('mustard', '<=', 0.5), ('baking powder', '<=', 0.5), ('rolls/buns', '<=', 0.5), ('white bread', '<=', 0.5), ('condensed milk', '<=', 0.5), ('flower soil/fertilizer', '<=', 0.5), ('decalcifier', '<=', 0.5), ('coffee', '<=', 0.5), ('misc. beverages', '<=', 0.5), ('artif. sweetener', '<=', 0.5), ('suga

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [70]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [71]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [72]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [73]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
tree_depth = 10
device = 'cpu'
use_cuda = device != 'cpu'

In [74]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [75]:
losses = []
accs = []
sparsity = []

In [76]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 025 | Total loss: 9.621 | Reg loss: 0.012 | Tree loss: 9.621 | Accuracy: 0.000000 | 6.428 sec/iter
Epoch: 00 | Batch: 001 / 025 | Total loss: 9.618 | Reg loss: 0.011 | Tree loss: 9.618 | Accuracy: 0.000000 | 6.431 sec/iter
Epoch: 00 | Batch: 002 / 025 | Total loss: 9.613 | Reg loss: 0.010 | Tree loss: 9.613 | Accuracy: 0.000000 | 6.465 sec/iter
Epoch: 00 | Batch: 003 / 025 | Total loss: 9.611 | Reg loss: 0.009 | Tree loss: 9.611 | Accuracy: 0.000000 | 6.474 sec/iter
Epoch: 00 | Batch: 004 / 025 | Total loss: 9.610 | Reg loss: 0.009 | Tree loss: 9.610 | Accuracy: 0.000000 | 6.494 sec/iter
Epoch: 00 | Batch: 005 / 025 | Total loss: 9.603 | Reg loss: 0.008 | Tree loss: 9.603 | Accuracy: 0.000000 | 6.525 sec/iter
Epoch: 00 | Batch: 006 / 025 | Total loss: 9.601 | Reg loss: 0.008 | Tree loss: 9.601 | Accuracy: 0.000000 | 6.486 

Epoch: 02 | Batch: 011 / 025 | Total loss: 9.527 | Reg loss: 0.007 | Tree loss: 9.527 | Accuracy: 0.085938 | 7.031 sec/iter
Epoch: 02 | Batch: 012 / 025 | Total loss: 9.527 | Reg loss: 0.007 | Tree loss: 9.527 | Accuracy: 0.082031 | 7.071 sec/iter
Epoch: 02 | Batch: 013 / 025 | Total loss: 9.523 | Reg loss: 0.007 | Tree loss: 9.523 | Accuracy: 0.072266 | 7.104 sec/iter
Epoch: 02 | Batch: 014 / 025 | Total loss: 9.521 | Reg loss: 0.008 | Tree loss: 9.521 | Accuracy: 0.076172 | 7.106 sec/iter
Epoch: 02 | Batch: 015 / 025 | Total loss: 9.520 | Reg loss: 0.008 | Tree loss: 9.520 | Accuracy: 0.064453 | 7.106 sec/iter
Epoch: 02 | Batch: 016 / 025 | Total loss: 9.518 | Reg loss: 0.008 | Tree loss: 9.518 | Accuracy: 0.085938 | 7.101 sec/iter
Epoch: 02 | Batch: 017 / 025 | Total loss: 9.517 | Reg loss: 0.009 | Tree loss: 9.517 | Accuracy: 0.042969 | 7.093 sec/iter
Epoch: 02 | Batch: 018 / 025 | Total loss: 9.510 | Reg loss: 0.009 | Tree loss: 9.510 | Accuracy: 0.060547 | 7.089 sec/iter
Epoch: 0

Epoch: 04 | Batch: 023 / 025 | Total loss: 9.343 | Reg loss: 0.017 | Tree loss: 9.343 | Accuracy: 0.070312 | 6.977 sec/iter
Epoch: 04 | Batch: 024 / 025 | Total loss: 9.333 | Reg loss: 0.018 | Tree loss: 9.333 | Accuracy: 0.068817 | 6.964 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 05 | Batch: 000 / 025 | Total loss: 9.423 | Reg loss: 0.012 | Tree loss: 9.423 | Accuracy: 0.078125 | 6.963 sec/iter
Epoch: 05 | Batch: 001 / 025 | Total loss: 9.420 | Reg loss: 0.012 | Tree loss: 9.420 | Accuracy: 0.089844 | 6.959 sec/iter
Epoch: 05 | Batch: 002 / 025 | Total loss: 9.409 | Reg loss: 0.012 | Tree loss: 9.409 | Accuracy: 0.076172 | 6.956 sec/iter
Epoch: 05 | Batch: 003 / 025 | Total loss: 9.396 | Reg loss: 0.012 | Tree loss: 9.396 | Ac

Epoch: 07 | Batch: 008 / 025 | Total loss: 9.049 | Reg loss: 0.019 | Tree loss: 9.049 | Accuracy: 0.058594 | 6.861 sec/iter
Epoch: 07 | Batch: 009 / 025 | Total loss: 9.045 | Reg loss: 0.019 | Tree loss: 9.045 | Accuracy: 0.091797 | 6.858 sec/iter
Epoch: 07 | Batch: 010 / 025 | Total loss: 9.014 | Reg loss: 0.019 | Tree loss: 9.014 | Accuracy: 0.099609 | 6.855 sec/iter
Epoch: 07 | Batch: 011 / 025 | Total loss: 8.995 | Reg loss: 0.020 | Tree loss: 8.995 | Accuracy: 0.091797 | 6.853 sec/iter
Epoch: 07 | Batch: 012 / 025 | Total loss: 8.980 | Reg loss: 0.020 | Tree loss: 8.980 | Accuracy: 0.076172 | 6.849 sec/iter
Epoch: 07 | Batch: 013 / 025 | Total loss: 8.960 | Reg loss: 0.020 | Tree loss: 8.960 | Accuracy: 0.085938 | 6.846 sec/iter
Epoch: 07 | Batch: 014 / 025 | Total loss: 8.937 | Reg loss: 0.021 | Tree loss: 8.937 | Accuracy: 0.076172 | 6.843 sec/iter
Epoch: 07 | Batch: 015 / 025 | Total loss: 8.921 | Reg loss: 0.021 | Tree loss: 8.921 | Accuracy: 0.078125 | 6.84 sec/iter
Epoch: 07

Epoch: 09 | Batch: 020 / 025 | Total loss: 8.342 | Reg loss: 0.027 | Tree loss: 8.342 | Accuracy: 0.095703 | 6.851 sec/iter
Epoch: 09 | Batch: 021 / 025 | Total loss: 8.291 | Reg loss: 0.027 | Tree loss: 8.291 | Accuracy: 0.105469 | 6.85 sec/iter
Epoch: 09 | Batch: 022 / 025 | Total loss: 8.311 | Reg loss: 0.027 | Tree loss: 8.311 | Accuracy: 0.070312 | 6.853 sec/iter
Epoch: 09 | Batch: 023 / 025 | Total loss: 8.267 | Reg loss: 0.028 | Tree loss: 8.267 | Accuracy: 0.074219 | 6.851 sec/iter
Epoch: 09 | Batch: 024 / 025 | Total loss: 8.271 | Reg loss: 0.028 | Tree loss: 8.271 | Accuracy: 0.094624 | 6.847 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 10 | Batch: 000 / 025 | Total loss: 8.588 | Reg loss: 0.023 | Tree loss: 8.588 | Acc

...

Epoch: 88 | Batch: 000 / 025 | Total loss: 3.147 | Reg loss: 0.029 | Tree loss: 3.147 | Accuracy: 0.091797 | 6.396 sec/iter
Epoch: 88 | Batch: 001 / 025 | Total loss: 3.002 | Reg loss: 0.029 | Tree loss: 3.002 | Accuracy: 0.130859 | 6.396 sec/iter
Epoch: 88 | Batch: 002 / 025 | Total loss: 3.073 | Reg loss: 0.029 | Tree loss: 3.073 | Accuracy: 0.095703 | 6.395 sec/iter
Epoch: 88 | Batch: 003 / 025 | Total loss: 3.028 | Reg loss: 0.029 | Tree loss: 3.028 | Accuracy: 0.117188 | 6.395 sec/iter
Epoch: 88 | Batch: 004 / 025 | Total loss: 3.129 | Reg loss: 0.029 | Tree loss: 3.129 | Accuracy: 0.105469 | 6.395 sec/iter
Epoch: 88 | Batch: 005 / 025 | Total loss: 3.047 | Reg loss: 0.029 | Tree loss: 3.047 | Accuracy: 0.126953 | 6.395 sec/iter
Epoch: 88 | Batch: 006 / 025 | Total loss: 3.059 | Reg loss: 0.029 | Tree loss: 3.059 | Accuracy: 0.089844 | 6.395 sec/iter
Epoch: 88 | Batch: 007 / 025 | Total loss: 3.020 | Reg loss: 0.029 | Tree loss: 3.020 | Accuracy: 0.099609 | 6.395 sec/iter
Epoch: 8

Epoch: 90 | Batch: 012 / 025 | Total loss: 2.963 | Reg loss: 0.029 | Tree loss: 2.963 | Accuracy: 0.128906 | 6.387 sec/iter
Epoch: 90 | Batch: 013 / 025 | Total loss: 3.012 | Reg loss: 0.029 | Tree loss: 3.012 | Accuracy: 0.107422 | 6.387 sec/iter
Epoch: 90 | Batch: 014 / 025 | Total loss: 2.970 | Reg loss: 0.029 | Tree loss: 2.970 | Accuracy: 0.095703 | 6.387 sec/iter
Epoch: 90 | Batch: 015 / 025 | Total loss: 3.032 | Reg loss: 0.029 | Tree loss: 3.032 | Accuracy: 0.091797 | 6.387 sec/iter
Epoch: 90 | Batch: 016 / 025 | Total loss: 3.002 | Reg loss: 0.029 | Tree loss: 3.002 | Accuracy: 0.115234 | 6.387 sec/iter
Epoch: 90 | Batch: 017 / 025 | Total loss: 2.931 | Reg loss: 0.029 | Tree loss: 2.931 | Accuracy: 0.125000 | 6.387 sec/iter
Epoch: 90 | Batch: 018 / 025 | Total loss: 2.991 | Reg loss: 0.029 | Tree loss: 2.991 | Accuracy: 0.097656 | 6.387 sec/iter
Epoch: 90 | Batch: 019 / 025 | Total loss: 2.960 | Reg loss: 0.029 | Tree loss: 2.960 | Accuracy: 0.105469 | 6.387 sec/iter
Epoch: 9

Epoch: 92 | Batch: 024 / 025 | Total loss: 2.929 | Reg loss: 0.029 | Tree loss: 2.929 | Accuracy: 0.105376 | 6.38 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 93 | Batch: 000 / 025 | Total loss: 3.094 | Reg loss: 0.028 | Tree loss: 3.094 | Accuracy: 0.134766 | 6.38 sec/iter
Epoch: 93 | Batch: 001 / 025 | Total loss: 3.079 | Reg loss: 0.028 | Tree loss: 3.079 | Accuracy: 0.107422 | 6.379 sec/iter
Epoch: 93 | Batch: 002 / 025 | Total loss: 3.107 | Reg loss: 0.028 | Tree loss: 3.107 | Accuracy: 0.089844 | 6.379 sec/iter
Epoch: 93 | Batch: 003 / 025 | Total loss: 3.082 | Reg loss: 0.028 | Tree loss: 3.082 | Accuracy: 0.103516 | 6.379 sec/iter
Epoch: 93 | Batch: 004 / 025 | Total loss: 3.015 | Reg loss: 0.028 | Tree loss: 3.015 | Accu

Epoch: 95 | Batch: 009 / 025 | Total loss: 3.056 | Reg loss: 0.028 | Tree loss: 3.056 | Accuracy: 0.093750 | 6.372 sec/iter
Epoch: 95 | Batch: 010 / 025 | Total loss: 3.006 | Reg loss: 0.028 | Tree loss: 3.006 | Accuracy: 0.105469 | 6.372 sec/iter
Epoch: 95 | Batch: 011 / 025 | Total loss: 2.986 | Reg loss: 0.028 | Tree loss: 2.986 | Accuracy: 0.109375 | 6.372 sec/iter
Epoch: 95 | Batch: 012 / 025 | Total loss: 3.008 | Reg loss: 0.028 | Tree loss: 3.008 | Accuracy: 0.072266 | 6.372 sec/iter
Epoch: 95 | Batch: 013 / 025 | Total loss: 3.000 | Reg loss: 0.028 | Tree loss: 3.000 | Accuracy: 0.091797 | 6.371 sec/iter
Epoch: 95 | Batch: 014 / 025 | Total loss: 3.014 | Reg loss: 0.028 | Tree loss: 3.014 | Accuracy: 0.082031 | 6.371 sec/iter
Epoch: 95 | Batch: 015 / 025 | Total loss: 2.980 | Reg loss: 0.028 | Tree loss: 2.980 | Accuracy: 0.132812 | 6.371 sec/iter
Epoch: 95 | Batch: 016 / 025 | Total loss: 2.960 | Reg loss: 0.028 | Tree loss: 2.960 | Accuracy: 0.125000 | 6.371 sec/iter
Epoch: 9

Epoch: 97 | Batch: 021 / 025 | Total loss: 3.009 | Reg loss: 0.028 | Tree loss: 3.009 | Accuracy: 0.085938 | 6.366 sec/iter
Epoch: 97 | Batch: 022 / 025 | Total loss: 2.968 | Reg loss: 0.028 | Tree loss: 2.968 | Accuracy: 0.099609 | 6.366 sec/iter
Epoch: 97 | Batch: 023 / 025 | Total loss: 2.915 | Reg loss: 0.028 | Tree loss: 2.915 | Accuracy: 0.082031 | 6.366 sec/iter
Epoch: 97 | Batch: 024 / 025 | Total loss: 2.906 | Reg loss: 0.028 | Tree loss: 2.906 | Accuracy: 0.092473 | 6.365 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 98 | Batch: 000 / 025 | Total loss: 3.064 | Reg loss: 0.028 | Tree loss: 3.064 | Accuracy: 0.107422 | 6.365 sec/iter
Epoch: 98 | Batch: 001 / 025 | Total loss: 3.066 | Reg loss: 0.028 | Tree loss: 3.066 | Ac

In [77]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [78]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 9.966666666666667


# Extract Rules

# Accumulate samples in the leaves

In [91]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 990


In [112]:
method = 'greedy'

In [113]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [114]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0

for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    if len(conds) == 0:
        continue
    
    comp = sum([cond.comprehensibility for cond in conds])
    print(f"============== Pattern {pattern_counter + 1} | comprehensibility: {comp} ==============")
    print(conds)
    print()
    print()
    print()
    sum_comprehensibility += comp
    
print(f"Average comprehensibility: {sum_comprehensibility / len(leaves)}")

12753
============== Pattern 46 | comprehensibility: 42 ==============
[-0.22315509617328644 * whole milk + -0.1576598435640335 * yogurt >= tensor(5.8347), -0.22315509617328644 * whole milk + -0.1576598435640335 * yogurt <= tensor(5.8356), 0.22214579582214355 * whole milk + 0.15752269327640533 * yogurt >= tensor(0.0029), 0.22214579582214355 * whole milk + 0.15752269327640533 * yogurt <= tensor(0.3817), -0.22095395624637604 * whole milk + -0.15674421191215515 * yogurt >= tensor(5.8491), -0.22095395624637604 * whole milk + -0.15674421191215515 * yogurt <= tensor(5.8500), 0.21899883449077606 * whole milk + 0.42691516876220703 * rolls/buns >= tensor(0.0030), 0.21899883449077606 * whole milk + 0.42691516876220703 * rolls/buns <= tensor(0.6474), -0.15359069406986237 * yogurt + -0.3081378936767578 * soda >= tensor(5.8474), -0.15359069406986237 * yogurt + -0.3081378936767578 * soda <= tensor(5.8485), -0.20576642453670502 * whole milk + -0.14803259074687958 * yogurt >= tensor(5.8460), -0.205766